In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, sum
from pyspark.ml.feature import StringIndexer
from pyspark.sql import DataFrame
from pyspark.sql.types import IntegerType

In [2]:

spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .master("spark://spark-master:7077") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()


In [4]:
df = spark.read.parquet("hdfs://namenode:9000/raw_data")

In [5]:
# Đếm tổng số dòng
total_rows = df.count()

# Đếm số dòng trùng lặp
duplicate_rows = df.groupBy(df.columns).count().filter("count > 1").count()

# Hiển thị kết quả
print(f"Tổng số dòng: {total_rows}")
print(f"Số dòng trùng lặp: {duplicate_rows}")


Tổng số dòng: 1000
Số dòng trùng lặp: 24


In [5]:
df.show()

+--------------------+----------+---------+--------+---------+--------------------+------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+--------------------+--------------------+------------------+------------------+------------------+
|          crash_date|crash_time|  borough|latitude|longitude|      on_street_name|   off_street_name|number_of_persons_injured|number_of_persons_killed|number_of_pedestrians_injured|number_of_pedestrians_killed|number_of_cyclist_injured|number_of_cyclist_killed|number_of_motorist_injured|number_of_motorist_killed|contributing_factor_vehicle_1|contributing_factor_vehicle_2|  vehicle_type_code1|  vehicle_type_code2|vehicle_type_code3|vehicle_type_code4|vehicle_type_code5|
+--------------------+----------+---------+-----

In [6]:
df.printSchema()

root
 |-- crash_date: string (nullable = true)
 |-- crash_time: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- on_street_name: string (nullable = true)
 |-- off_street_name: string (nullable = true)
 |-- number_of_persons_injured: integer (nullable = true)
 |-- number_of_persons_killed: integer (nullable = true)
 |-- number_of_pedestrians_injured: integer (nullable = true)
 |-- number_of_pedestrians_killed: integer (nullable = true)
 |-- number_of_cyclist_injured: integer (nullable = true)
 |-- number_of_cyclist_killed: integer (nullable = true)
 |-- number_of_motorist_injured: integer (nullable = true)
 |-- number_of_motorist_killed: integer (nullable = true)
 |-- contributing_factor_vehicle_1: string (nullable = true)
 |-- contributing_factor_vehicle_2: string (nullable = true)
 |-- vehicle_type_code1: string (nullable = true)
 |-- vehicle_type_code2: string (nullable = true)
 |-- veh

In [7]:
df = df.withColumn('is_injured',
                   when((col('number_of_persons_injured').cast(IntegerType()) +
                         col('number_of_persons_killed').cast(IntegerType())) > 0, 1).otherwise(0))

In [8]:
# Xoa nhung cot khong can thiet
cols_to_drop = ['latitude', 'longitude', 'on_street_name', 'off_street_name', 'number_of_persons_injured', 'number_of_pedestrians_killed',
                'number_of_cyclist_injured', 'number_of_motorist_injured', 'number_of_motorist_killed',
                'vehicle_type_code3', 'vehicle_type_code4', 'vehicle_type_code5',
                'crash_date', 'crash_time', 'number_of_persons_killed', 'number_of_pedestrians_injured',
                'number_of_cyclist_killed']
df_dropped_cols = df.drop(*cols_to_drop)

In [9]:
#Kiem tra du lieu thieu
missing_counts = df_dropped_cols.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_dropped_cols.columns])
missing_counts.show()

+-------+-----------------------------+-----------------------------+------------------+------------------+----------+
|borough|contributing_factor_vehicle_1|contributing_factor_vehicle_2|vehicle_type_code1|vehicle_type_code2|is_injured|
+-------+-----------------------------+-----------------------------+------------------+------------------+----------+
|     34|                            0|                           56|                 0|                67|         0|
+-------+-----------------------------+-----------------------------+------------------+------------------+----------+



In [10]:
#Fill du lieu thieu bang gia tri mode
categorical_cols = ['contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
                    'vehicle_type_code1', 'vehicle_type_code2', 'borough']

# for col_name in categorical_cols:
#     # Get the mode value and bring it to the driver
#     mode_value = df_dropped_cols.groupBy(col_name).count().orderBy('count', ascending=False).first()[0]
#     # Fill missing values with the mode
#     df_dropped_cols = df_dropped_cols.fillna({col_name: mode_value})
for col_name in categorical_cols:
    # Lấy mode
    mode_row = df_dropped_cols.groupBy(col_name).count().orderBy('count', ascending=False).first()
    
    # Kiểm tra nếu mode_row và mode_value không phải là None
    if mode_row is not None and mode_row[0] is not None:
        mode_value = mode_row[0]
    else:
        # Nếu không có mode, sử dụng giá trị 0
        mode_value = 0
    
    # Điền giá trị thiếu bằng mode hoặc 0
    df_dropped_cols = df_dropped_cols.fillna({col_name: mode_value})


In [11]:
#Ma hoa du lieu bien phan loai
def encode_categorical_cols(df: DataFrame, categorical_cols: list) -> DataFrame:
    for col_name in categorical_cols:
        indexer = StringIndexer(inputCol=col_name, outputCol=col_name + "_encoded", handleInvalid='keep')
        df = indexer.fit(df).transform(df)
    return df

df_encoded = encode_categorical_cols(df_dropped_cols, categorical_cols)

In [12]:
df_final = df_encoded.drop(*categorical_cols)

In [14]:
# Kiểm tra lại lần nữa xem còn giá trị null không
missing_counts = df_final.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_final.columns])
missing_counts.show()

+----------+-------------------------------------+-------------------------------------+--------------------------+--------------------------+---------------+
|is_injured|contributing_factor_vehicle_1_encoded|contributing_factor_vehicle_2_encoded|vehicle_type_code1_encoded|vehicle_type_code2_encoded|borough_encoded|
+----------+-------------------------------------+-------------------------------------+--------------------------+--------------------------+---------------+
|         0|                                    0|                                    0|                         0|                         0|              0|
+----------+-------------------------------------+-------------------------------------+--------------------------+--------------------------+---------------+



In [15]:
#Nếu còn dữ liệu null thì xóa các dòng đó đi
df_final = df_final.drop(how ='any', thresh=None, subset=None)
df_final.toPandas().info()

TypeError: DataFrame.drop() got an unexpected keyword argument 'how'

In [16]:
df_final.show()

+----------+-------------------------------------+-------------------------------------+--------------------------+--------------------------+---------------+
|is_injured|contributing_factor_vehicle_1_encoded|contributing_factor_vehicle_2_encoded|vehicle_type_code1_encoded|vehicle_type_code2_encoded|borough_encoded|
+----------+-------------------------------------+-------------------------------------+--------------------------+--------------------------+---------------+
|         0|                                  1.0|                                  2.0|                       0.0|                       1.0|            1.0|
|         0|                                  0.0|                                  0.0|                       0.0|                       1.0|            1.0|
|         0|                                  6.0|                                  0.0|                       5.0|                       2.0|            1.0|
|         0|                                  

In [17]:
#Đưa ra thống kê cơ bản
df_final.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
is_injured,198,0.0,0.0,0,0
contributing_factor_vehicle_1_encoded,198,2.1666666666666665,3.190643045795828,0.0,13.0
contributing_factor_vehicle_2_encoded,198,0.3686868686868687,1.042463546906425,0.0,5.0
vehicle_type_code1_encoded,198,0.8888888888888888,1.3659298083718503,0.0,6.0
vehicle_type_code2_encoded,198,1.9696969696969697,2.5903159968513627,0.0,11.0
borough_encoded,198,0.6919191919191919,0.9129832583268603,0.0,4.0


In [18]:
# feature
x = df_final.drop("is_injured")
# label
y = df_final.select("is_injured")